# Dask Array

Материалы:
* Макрушин С.В. Лекция 11: Dask
* https://docs.dask.org/en/latest/array.html
* JESSE C. DANIEL. Data Science with Python and Dask. 

In [1]:
import dask.array as da
import numpy as np
import h5py

## Задачи для совместного разбора

1. Создайте массив размерностью 1000 на 300000, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики полученного массива.

In [2]:
arr_np = np.random.normal(0, 1, size=(1000, 300000))
arr_np

array([[-1.11191799, -1.03731836,  0.66191624, ...,  0.28652401,
         0.22788838,  0.502246  ],
       [ 0.95895796,  0.17217181,  0.38510215, ..., -1.05551292,
        -1.06005381,  0.43714536],
       [ 0.61508178, -1.87744323,  0.24714738, ...,  0.57272789,
         1.03984061,  0.42694402],
       ...,
       [ 0.5306254 , -0.58159694,  2.26058301, ..., -0.10366599,
         0.38271166,  1.9676833 ],
       [-0.65867717,  1.87710977,  0.57082884, ..., -0.65471681,
         0.20648696,  0.67856711],
       [-1.18507251,  0.50721931, -0.86171815, ..., -0.71135995,
         2.19610118, -1.1146102 ]])

In [3]:
arr_da = da.random.normal(0, 1, size=(1000, 300000), chunks=(1000, 300000))
arr_da

C:\ProgramData\Anaconda3\lib\site-packages\dask\array\core.py:1401: RuntimeWarning: overflow encountered in long_scalars
  cbytes = format_bytes(np.prod(self.chunksize) * self.dtype.itemsize)


dask.array<normal, shape=(1000, 300000), dtype=float64, chunksize=(1000, 300000), chunktype=numpy.ndarray>

2. Посчитайте сумму квадратов элементов массива, созданного в задаче 1. Создайте массив `np.array` такого же размера и сравните скорость решения задачи с использование `da.array` и `np.array`

In [4]:
%%time
np.power(arr_np, 2).sum()

Wall time: 8.92 s


300011877.46912813

In [5]:
%%time
da.power(arr_da, 2).sum().compute()

Wall time: 17.4 s


300012153.49111545

3. Визуализируйте граф вычислений для задачи 2.

In [6]:
#da.power(arr_da, 2).sum().visualize()

## Лабораторная работа 11

1. Считайте датасет `recipe` из файла `minutes_n_ingredients_full.hdf5` в виде `dask.array`. Укажите аргумент `chunks=(100_000, 3)` при создании массива. Выведите на экран основную информацию о массиве.

In [7]:
hdf = h5py.File('minutes_n_ingredients_full.hdf5', 'r')
recipe = da.from_array(hdf['recipe'], chunks=(100000, 3))

In [8]:
recipe

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(100000, 3), chunktype=numpy.ndarray>

2. Вычислите среднее значение по каждому столбцу, кроме первого. 

In [9]:
recipe[:, 1:].mean(axis=0).compute()

array([1004.20805176,    5.4198008 ])

3. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

In [10]:
%%time
recipe[:, 1:].mean(axis=0).compute()

Wall time: 77.1 ms


array([1004.20805176,    5.4198008 ])

In [11]:
%%time
recipe1 = da.from_array(hdf['recipe'], chunks=(1000, 3))
recipe1[:, 1:].mean(axis=0).compute()

Wall time: 1.63 s


array([1004.20805176,    5.4198008 ])

4. Выберите рецепты, время выполнения которых меньше медианного значения

In [12]:
da.median(recipe[:, 1], axis=0).compute()

32.0

In [13]:
recipe[recipe[:, 1] < da.median(recipe[:, 1], axis=0)].compute()

array([[1089012,      23,       5],
       [1428572,       0,       5],
       [1400250,      24,       1],
       ...,
       [1029131,      19,       4],
       [1700703,       1,       1],
       [ 713836,       0,       9]], dtype=int64)

5. Посчитайте количество каждого из возможных значений кол-ва ингредиентов

In [14]:
cnt = da.bincount(recipe[:, 2]).compute()
i = np.nonzero(cnt)[0]
da.vstack((i, cnt[i])).T.compute()

array([[     1, 222071],
       [     2, 224158],
       [     3, 229388],
       [     4, 234948],
       [     5, 240720],
       [     6, 244360],
       [     7, 247181],
       [     8, 246747],
       [     9, 246816],
       [    10,  22430],
       [    11,  19094],
       [    12,  15165],
       [    13,  11640],
       [    14,   8284],
       [    15,   6014],
       [    16,   4145],
       [    17,   2793],
       [    18,   1913],
       [    19,   1279],
       [    20,    852],
       [    21,    529],
       [    22,    346],
       [    23,    244],
       [    24,    178],
       [    25,    107],
       [    26,     68],
       [    27,     55],
       [    28,     33],
       [    29,     22],
       [    30,     20],
       [    31,     13],
       [    32,      5],
       [    33,      4],
       [    34,      3],
       [    35,      4],
       [    36,      1],
       [    37,      2],
       [    38,      1],
       [    39,      1],
       [    40,      2],


6. Найдите максимальную продолжительность рецепта. Ограничьте максимальную продолжительность рецептов сверху значением, равному 75% квантилю.

In [15]:
max_time = recipe[recipe[:, 1] == recipe[:, 1].max(axis=0), 1].compute()
max_time

array([2147483647], dtype=int64)

In [16]:
quan = da.percentile(recipe[:, 1], q =75).compute()
quan

array([49.])

In [17]:
rec = recipe.compute()
rec[:, 1]  = da.where(recipe[:, 1] < quan, recipe[:, 1], quan).compute()
recipe = da.array(rec)

In [18]:
recipe.compute() 

array([[ 683970,      33,       9],
       [1089012,      23,       5],
       [1428572,       0,       5],
       ...,
       [1910650,      49,       2],
       [ 713836,       0,       9],
       [ 660699,      49,       8]], dtype=int64)

7. Создайте массив `dask.array` из 2 чисел, содержащих ваши предпочтения относительно времени выполнения рецепта и кол-ва ингредиентов. Найдите наиболее похожий (в смысле $L_1$) рецепт из имеющихся в датасете.

In [19]:
pref = da.array([15, 6])
recipe[(abs(da.linalg.norm(recipe[:, 1:], 1, axis=1) - da.linalg.norm(pref, 1))).argmin()].compute() 

array([1132618,      15,       6], dtype=int64)

8. Работая с исходным файлом в формате `hdf5`, реализуйте алгоритм подсчета среднего значения в блочной форме и вычислите с его помощью среднее значение второго столбца в массиве.

Блочный алгоритм вычислений состоит из двух частей:
1. загрузка фрагмента за фрагментом данных по `blocksize` элементов и проведение вычислений на этим фрагментом;
2. агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных.

Важно: при работе с `h5py` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент

In [20]:
def mean_hd(arr):
    return arr.sum(), len(arr)

In [21]:
def mean_merge(dts, blocksize):
    s = 0
    res = []
    while (s < dts.len() - 1):
        f = s + blocksize
        res.append(mean_hd(dts[s:f, 1]))
        s = f
    res = da.array(res)
    return res[:, 0].sum()/res[:, 1].sum()

In [22]:
mean_merge(hdf['recipe'], 100).compute()

1004.2080517575215